# Z Estimator

## Importing libraries

In [91]:
# Libraries to help with reading and manipulating data
import numpy as np
import gc
import pandas as pd
from ydata_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

## Extracting the dataset

In [92]:
df_train_prop2016 = pd.read_csv('properties_2016.csv')
df_train_prop2017 = pd.read_csv('properties_2017.csv')
df_train_prop = pd.concat([df_train_prop2016, df_train_prop2017])
df_train_prop.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [93]:
print(r'The dataset has {rownum} rows and {colnum} columns'.format(rownum = df_train_prop.shape[0], colnum = df_train_prop.shape[1]))

The dataset has 5970434 rows and 58 columns


In [94]:
df_train_2016 = pd.read_csv('train_2016_v2.csv')
df_train_2017 = pd.read_csv('train_2017.csv')
df_train = pd.concat([df_train_2016, df_train_2017])
df_train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [95]:
print(r'The dataset has {rownum} rows and {colnum} columns'.format(rownum = df_train_2016.shape[0], colnum = df_train_2017.shape[1]))

The dataset has 90275 rows and 3 columns


In [96]:
df = pd.merge(df_train_prop,   df_train,  on="parcelid", how="inner")

In [97]:
print(r'The dataset has {rownum} rows and {colnum} columns'.format(rownum = df_train.shape[0], colnum = df_train.shape[1]))

The dataset has 167888 rows and 3 columns


In [98]:
# Delete the old DataFrame
del [df_train_prop, df_train, df_train_2016, df_train_2017, df_train_prop2016, df_train_prop2017]
 
# Perform garbage collection
gc.collect()

3233

## Data preprocessing

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335776 entries, 0 to 335775
Data columns (total 60 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   parcelid                      335776 non-null  int64  
 1   airconditioningtypeid         107623 non-null  float64
 2   architecturalstyletypeid      936 non-null     float64
 3   basementsqft                  186 non-null     float64
 4   bathroomcnt                   335601 non-null  float64
 5   bedroomcnt                    335601 non-null  float64
 6   buildingclasstypeid           63 non-null      float64
 7   buildingqualitytypeid         214377 non-null  float64
 8   calculatedbathnbr             332216 non-null  float64
 9   decktypeid                    2548 non-null    float64
 10  finishedfloor1squarefeet      25851 non-null   float64
 11  calculatedfinishedsquarefeet  334084 non-null  float64
 12  finishedsquarefeet12          319156 non-nul

In [100]:
df.isnull().sum() / df.shape[0] * 100.00

parcelid                         0.000000
airconditioningtypeid           67.947977
architecturalstyletypeid        99.721243
basementsqft                    99.944606
bathroomcnt                      0.052118
bedroomcnt                       0.052118
buildingclasstypeid             99.981237
buildingqualitytypeid           36.154758
calculatedbathnbr                1.060231
decktypeid                      99.241161
finishedfloor1squarefeet        92.301117
calculatedfinishedsquarefeet     0.503907
finishedsquarefeet12             4.949728
finishedsquarefeet13            99.955327
finishedsquarefeet15            96.076551
finishedsquarefeet50            92.301117
finishedsquarefeet6             99.522301
fips                             0.052118
fireplacecnt                    89.333365
fullbathcnt                      1.060231
garagecarcnt                    66.944630
garagetotalsqft                 66.944630
hashottuborspa                  97.682681
heatingorsystemtypeid           37

There a huge percentage of missing values in the columns

In [101]:
# Filling the missing values with a new id (Unknown) based on the data dictionary
df = df.fillna({'airconditioningtypeid': 14,
                'architecturalstyletypeid': 28,
                'buildingclasstypeid': 6,
                'heatingorsystemtypeid' : 26,
                'storytypeid' : 36,
                'typeconstructiontypeid' : 19
                })

In [102]:
df.drop(['decktypeid', 'calculatedbathnbr', 'fips', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'propertyzoningdesc',
         'basementsqft', 'finishedfloor1squarefeet', 
         'rawcensustractandblock', 'propertycountylandusecode', 'censustractandblock', 'parcelid'], axis=1, inplace=True)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
# Enconding booleans as 1 or 0
booleanColumns = ['hashottuborspa', 'fireplaceflag', 'taxdelinquencyflag']
for column in booleanColumns:
    df[column] = df[column].apply(lambda x: 1 if x == True or x == 'Y' else 0)

In [ ]:
# Checking the years of deliquency depending on the flag
df.groupby(['taxdelinquencyyear', 'taxdelinquencyflag']).size()

- It shows only years when the flag is 1, it can be filled the missing ones with 0

In [78]:
# Filling with 0
df = df.fillna({'taxdelinquencyyear': 0})

In [83]:
# # Adding the median for missing values in 'Mileage", "Engine", "Power", grouping by Manufacturer
# df[["basementsqft", "bathroomcnt", "bedroomcnt"]] = df.groupby(['airconditioningtypeid', 'architecturalstyletypeid', 'buildingclasstypeid', 'heatingorsystemtypeid',
#                                                       'latitude', 'longitude', 'propertylandusetypeid', 'storytypeid', 'fireplaceflag', 'taxdelinquencyflag', 
#                                                       'taxdelinquencyyear', 'transactiondate'
#                                                       ])[["basementsqft", "bathroomcnt", "bedroomcnt"]].transform(lambda x: x.fillna(x.median()))

In [105]:
df.isnull().sum() / df.shape[0] * 100.00

airconditioningtypeid            0.000000
architecturalstyletypeid         0.000000
bathroomcnt                      0.052118
bedroomcnt                       0.052118
buildingclasstypeid              0.000000
buildingqualitytypeid           36.154758
finishedfloor1squarefeet        92.301117
calculatedfinishedsquarefeet     0.503907
finishedsquarefeet12             4.949728
finishedsquarefeet13            99.955327
finishedsquarefeet15            96.076551
finishedsquarefeet50            92.301117
finishedsquarefeet6             99.522301
fireplacecnt                    89.333365
fullbathcnt                      1.060231
garagecarcnt                    66.944630
garagetotalsqft                 66.944630
hashottuborspa                  97.682681
heatingorsystemtypeid            0.000000
latitude                         0.052118
longitude                        0.052118
lotsizesquarefeet               10.995723
poolcnt                         79.703135
poolsizesum                     98

In [74]:
df["transactiondate"] = pd.to_datetime(df["transactiondate"], format="%Y-%m-%d")

## Exploratory Data Analysis